In [0]:
PCA_GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/rbf/bosz/bosz_5000_full_3/pca_2'
RBF_GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/rbf/bosz/bosz_5000_full_3/pca-rbf_4'

In [0]:
%matplotlib inline

In [0]:
import sys

# Allow load project as module
sys.path.insert(0, '../../../..')

In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

In [0]:
import matplotlib     
matplotlib.rcParams.update({'font.size': 14})

# Load stellar model grid

In [0]:
from pfsspec.core.grid import ArrayGrid
from pfsspec.core.grid import RbfGrid
from pfsspec.stellar.grid import ModelGrid
from pfsspec.stellar.grid.bosz import Bosz
from pfsspec.stellar.grid.phoenix import Phoenix

In [0]:
fn = os.path.join(PCA_GRID_PATH, 'spectra.h5')
pca_grid = ModelGrid(Bosz(pca=True, normalized=True), ArrayGrid)
pca_grid.preload_arrays = False
pca_grid.load(fn, format='h5')

In [0]:
fn = os.path.join(RBF_GRID_PATH, 'spectra.h5')
rbf_grid = ModelGrid(Bosz(pca=True, normalized=True), RbfGrid)
rbf_grid.preload_arrays = False
rbf_grid.load(fn, format='h5')

In [0]:
axes = rbf_grid.get_axes()
for k in axes.keys():
    print(k, axes[k].values)

In [0]:
rbf_grid.grid.eigs.keys()

In [0]:
rbf_grid.wave.shape, rbf_grid.grid.eigv['flux'].shape

# Plot

In [0]:
M_H = 0
T_eff = 4500
log_g = 1
a_M = 0
C_M = 0

idx = rbf_grid.grid.get_nearest_index(M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M, C_M=C_M)
idx

In [0]:
M_H = -2.5
T_eff = 3500
log_g = 0.0
a_M = -0.25
C_M = -0.75

idx = rbf_grid.grid.get_nearest_index(M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M, C_M=C_M)
idx

In [0]:
M_H = -1.15
T_eff = 5800
log_g = 2.71
a_M = -0.11
C_M = -0.0

idx = rbf_grid.grid.get_nearest_index(M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M, C_M=C_M)
idx

In [0]:
model = rbf_grid.get_model(M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M, C_M=C_M)
plt.plot(model.wave, model.flux)

In [0]:
model = rbf_grid.get_model(M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M, C_M=C_M, denormalize=True)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(6, 4), dpi=120)

ax.plot(model.wave, model.flux, lw=0.1)

#plt.ylim(0, 1e6)

# Plot PCs

In [0]:
M_H = 0
T_eff = 4500
log_g = 1
a_M = 0
C_M = 0

rbf_idx = rbf_grid.grid.get_nearest_index(M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M, C_M=C_M)
rbf_idx

In [0]:
array_idx = pca_grid.grid.get_nearest_index(M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M, C_M=C_M)
array_idx

In [0]:
def plot_params_hires(rbf_grid, rbf, idx=2, param_idx=0):
    # idx: iterate through this dimension in subplots
    # param_idx: which parameter to plot (last dimension of the arrays)
    
    axes = [ rbf_grid.axes[k] for k in rbf_grid.axes ]
    pp = axes[idx].values.size
    rr = int(pp / 4 + 0.5)
    f, axs = plt.subplots(rr, 4, figsize=(16, 4 * rr))
    for p in range(pp):
        i = p // 4
        j = p % 4
        
        coords = [ np.linspace(0, axes[i].values.size - 1, 20) for i in range(len(axes)) ]

        coords[idx] = np.array([p])
        coords[-2] = np.array([array_idx[-2]])
        coords[-1] = np.array([array_idx[-1]])
        
        coords = np.meshgrid(*coords, indexing='ij')
        
        rbf_values = rbf(*[ x.flatten() for x in coords])
        rbf_values = rbf_values[..., param_idx].reshape(coords[0].shape)
        rbf_values = rbf_values.squeeze()
                
        image = rbf_values
        vmin, vmax = None, None
              
        l = axs[i, j].imshow(image, aspect='auto', vmin=vmin, vmax=vmax)
        f.colorbar(l, ax=axs[i, j])
        axs[i, j].set_xlabel('param: {} | slice: {}'.format(param_idx, p))

In [0]:
rbf = rbf_grid.rbf_grid.values['flux']
rbf, rbf.nodes.shape[-1]

In [0]:
for pi in range(5):
    plot_params_hires(rbf_grid.rbf_grid, rbf, param_idx=pi)